PREDICCIÓN DE PRECIOS DE MONEDAS DIGITALES CON MACHINE LEARNING, APRENDIZAJE AUTOMÁTICO PROFUNDO.

#DEFINICIÓN DEL PROBLEMA

El problema es sobre la desconfianza que existe en los mercados financieros tradicionales que nace debido a las diversas crisis financieras, intervenidas por la mano del hombre y que vienen afectando a la economía global desde el año 2007 (Crisis Subprime) y que probablemente seguirán ocurriendo en futuro. Por este motivo se ha producido el desvanecimiento del optimismo económico, la intensificación de las vulnerabilidades financieras y el deterioro de la confianza en los mercados tradicionales que ofrecen productos financieros altamente sofisticados, complejos y automatizados. Donde la mayoría de los inversionistas no conocen la naturaleza última de las operaciones contratadas o que tipos de tecnologías como el Blockchain pueden beneficiar tanto al inversor como a la banca.
La incertidumbre y la volatilidad en los mercados cada vez se incrementan y la inteligencia artificial (IA), el aprendizaje automático (ML) y el aprendizaje profundo (DL) han transformado las finanzas y la inversión. Lo que hace posible realizar modelos que permitan proyectar situaciones inciertas, como la predicción de precios que tendrán las monedas digitales o criptomonedas, acciones o cualquier activo financiero en los mercados. ¡"La Inteligencia artificial es para el trading lo que el fuego era para los hombres de las cavernas"!.
En este proyecto, entrenaremos un modelo de red neuronal profunda para predecir los precios futuros de las monedas digitales o criptomonedas.
Utilizaremos un tipo de red neuronal conocida como Long Short-Term Memory Networks (LSTM). Este modelo se entrenará utilizando datos históricos de precios de cierre de monedas digitales. 

# Metodología Proceso KDD

In [ ]:
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
#!apt-get update
!pip install yfinance
!pip install yahoofinancials

# 1. Recopilación de Datos

# 1.1 Instalación e importación de librerías

In [ ]:
# Biblioteca de Importación
# a)	Manipulación y tratamiento de datos.
import numpy as np
import pandas as pd
import math
import datetime as dt
import os
import seaborn as sns
import pandas_datareader as pdr
import requests
import plotly.figure_factory as ff
from pandas_datareader import data
import yfinance as yf

# b)	Visualización de datos.
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import cycle
plt.style.use ('ggplot')
%matplotlib inline

# c)	Construcción de modelos usaremos estas Bibliotecas.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense, Dropout
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# d)	Evaluación.
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import accuracy_score
from statsmodels.tools.eval_measures import rmse
from sklearn import metrics

# e)	No presentar advertencias.
import warnings
# f)	Otras Necesarias
from copy import copy
from scipy import stats

In [ ]:
# Ingresar el Simbolo y los parámetros de la moneda digital Ej: BTC-USD,XRP-USD,ETH-BTC,JPM,AAPL,EURUSD=X,^IXIC,^GSPC,MSFT
symbol = 'BTC-USD'
stock= symbol
interval = '1day' # Supports: 1min, 5min, 15min, 30min, 45min, 1h, 2h, 4h, 1day, 1week, 1month
order = 'asc'
start_date = '2000-01-01'
end_date = '2022-12-01'
Act_fin=yf.download(stock,start=start_date,end=end_date, progress=False)

In [ ]:
Act_fin

In [ ]:
# Resetenado el index de la columna Fecha.
Act_fin= Act_fin.reset_index()

In [ ]:
# Renombrando las columnas 
df = Act_fin.rename(columns={'Date':'datetime','Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'})
# Ordenar los datos según la fecha
df = df.sort_values('datetime',ascending=True)
df.head()

In [ ]:
# Describe que tipo de datos existen en las columnas, variables o elemento de la matriz.
df.dtypes

#2. Limpieza,Pre-procesamiento y Análisis exploratorio-EDA(Exploratory Data Analysis).

#2.1 Limpieza de datos, comprobación de valores nulos

La limpieza de datos: se define como la eliminación de datos ruidosos e irrelevantes.
Limpieza en caso de valores faltantes.
Limpieza de datos ruidosos, donde el ruido es un error aleatorio o de varianza.
Entre otros.


In [ ]:
print('Total número de días presentes en el dataset: ',df.shape[0])
print('Total número de Variables o Columnas presentes en el dataset: ',df.shape[1])

In [ ]:
# Chekeando si existen valores nulos en los datos
df.isnull().sum()
print('Valores Nulos a Eliminar:',df.isnull().values.sum())
print('Valores NA a Eliminar:',df.isnull().values.any())
# Eliminando Valores Nulos, descartamos todos los valores nulos presentes en el conjunto de datos.
df=df.dropna()
print('Valores Nulos Remanentes:',df.isnull().values.sum())
print('Valores NA:',df.isnull().values.any())

In [ ]:
# Forma final del conjunto de datos después de tratar con valores nulos
df.shape

# 2.2 Pre-procesamiento de datos

In [ ]:
# Analizamos que tipos de datos existen para posteriormente tratarlos
df.info()

In [ ]:
#Por lo general la columna datetime es de tipo object (texto), para asegurarnos la convertiremos en formato de tiempo (Fecha y hora).
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')
df_y = df.loc[(df['datetime'] >= start_date)
                     & (df['datetime'] <= end_date)]

In [ ]:
#Eliminar la Columna Precio Ajustado y Volumen
df_y.drop(df_y[['Adj Close','volume']],axis=1)

In [ ]:
# Se convirtió el conjunto de datos a Mes
df_mes= df_y.groupby(df_y['datetime'].dt.strftime('%B'))[['open','close']].mean(numeric_only=True)
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
# Asignando el Index a la Fecha 
df_mes= df_mes.reindex(new_order, axis=0)
df_mes

In [ ]:
# Chekeando si existen valores nulos en los datos
df_mes.isnull().sum()
print('Valores Nulos a Eliminar:',df_mes.isnull().values.sum())
print('Valores NA a Eliminar:',df_mes.isnull().values.any())
# Eliminando Valores Nulos, descartamos todos los valores nulos presentes en el conjunto de datos.
df_mes=df_mes.dropna()
print('Valores Nulos Remanentes:',df_mes.isnull().values.sum())
print('Valores NA:',df_mes.isnull().values.any())


In [ ]:
#Definimos una funcios para normalizar los precios de las monedas digitales en función de su precio
def normalizacion(df):
  x = df.copy()
  # Se crea un ciclo for que recorra cada columna en data, excluyendo la columna de fecha, para comenzar desde el índice [1:], es decir, de la columna uno en adelante.
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x

In [ ]:
# Función para trazar gráficos interactivos usando Plotly Express
def ploteo_inter(df, title):
  fig = px.line(title = title)
#A continuación, se creará un bucle for para recorrer cada columna de nuestro marco de datos, excepto el primero, 
#y se tomará el objeto de fig.add_scatter para que comience a agregarle dispersiones, para que se pueda añadir frases a la misma. Así que voy a decir figura esa dispersión en el eje x. 
  for i in df.columns[1:]:
    fig.add_scatter(x = df['datetime'], y = df[i], name = i)
  fig.show()

In [ ]:
# Función para trazar gráficos interactivos con mayores atributos
def ploteo_linea(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel(symbol, fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);

In [ ]:
# Función para evaluar las metricas generales obtenidos del modelo
def evaluacion_metrica(X_true, X_pred, y_true, y_pred):
 
    def mean_absolute_percentage_error(X_true, X_pred, y_true, y_pred): 
        X_true, X_pred, y_true, y_pred, y_true, y_pred = np.array(X_true), np.array(X_pred),np.array(y_true), np.array(y_pred)
        return np.mean(np.abs(y_true - y_pred) / y_true) * 100
        return np.mean(np.abs(X_true - X_pred) / X_true) * 100
    print('Resultados de la métrica de evaluación:-')
    print("-------------------------------------------------------------------------------------")
    print('Métricas de evaluación RMSE, MSE y MAE:-')
    print('Train data RMSE es : ', math.sqrt(mean_squared_error(X_true,X_pred)))
    print('Train data MSE es : ', mean_squared_error(X_true,X_pred))
    print('Train data MAE es : ', mean_absolute_error(X_true,X_pred))
    print("Test data RMSE es : ", math.sqrt(mean_squared_error(y_true,y_pred)))
    print("Test data MSE es : ", mean_squared_error(y_true,y_pred))
    print("Test data MAE es : ", mean_absolute_error(y_true,y_pred))
    print("-------------------------------------------------------------------------------------")
    print('R square score for regression:-')
    print("Train data R2 score es :", r2_score(X_true, X_pred))
    print("Test data R2 score es:", r2_score(y_true, y_pred))
    print("-------------------------------------------------------------------------------------")
    print('Mean Gamma deviance and Mean Poisson deviance:-')
    print(f'Train data MGD es : , {metrics.mean_gamma_deviance(X_true, X_pred)}')
    print(f'Train data MGD es : , {metrics.mean_gamma_deviance(y_true, y_pred)}')
    print("Train data MGD es: ", mean_gamma_deviance(X_true, X_pred))
    print("Test data MGD es : ", mean_gamma_deviance(y_true, y_pred))
    print("Train data MPD es: ", mean_poisson_deviance(X_true, X_pred))
    print("Test data MPD es : ", mean_poisson_deviance(y_true, y_pred))

In [ ]:
#Se separa la fecha de entrenamiento (serie separada) para futuros trazados (mas adelante la usaremos como nuestro eje X)
df_fechas = df['datetime']
#Consultar últimas 15 fechas 
print(df_fechas.tail(15)) 

In [ ]:
# Visualización de Fechas y sus Precio de Apertura
df_fechas_cierre = df[['datetime','close']]
df_fechas_cierre 

In [ ]:
#Columnas (cols) con varias variables del precio para el entrenamiento
cols = list(df)[1:5]#['Open', 'High', 'Low', 'Close']
cols
df_cols= df[cols]
df_cols

In [ ]:
df_apertura = df['open']
print("Forma del precio de Apertura dataframe:", df_apertura.shape)

In [ ]:
df_cierre = df['close']
print("Forma del precio de Cierre dataframe:", df_cierre.shape)

# 2.3 Análisis exploratorio-EDA(Exploratory Data Analysis).

In [ ]:
df_fechas_cierre.describe().T

Se puede observar que el precio promedio de cierre del Activo durante el período de tiempo importado es de 12.819 USD
El Precio Máximo es 67.566 USD y su precio mínimo fue de 178 USD.


In [ ]:
df.describe().T

In [ ]:
# Gráficos interactivos para los datos
ploteo_inter(df_fechas_cierre, 'Precios de Cierre')
ploteo_inter(normalizacion(df_fechas_cierre),'Normalización de los Precios de Cierre')

In [ ]:
# Análisis Año 2021
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')
# Ingresar rango del año
y_2021 = df.loc[(df['datetime'] >= '2021-01-01')
                     & (df['datetime'] < '2021-12-31')]
mensual= y_2021.groupby(y_2021['datetime'].dt.strftime('%B'))[['open','close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
mensual = mensual.reindex(new_order, axis=0)
mensual

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=mensual.index,
    y=mensual['open'],
    name='Precio de Apertura',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=mensual.index,
    y=mensual['close'],
    name='Precio de Cierre',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Comparación mensual entre el precio de apertura y cierre Año 2021')
fig.show()

y_2021.groupby(y_2021['datetime'].dt.strftime('%B'))['low'].min()
mensual_h = y_2021.groupby(df['datetime'].dt.strftime('%B'))['high'].max()
mensual_h = mensual_h.reindex(new_order, axis=0)

mensual_l = y_2021.groupby(y_2021['datetime'].dt.strftime('%B'))['low'].min()
mensual_l = mensual_l.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=mensual_h.index,
    y=mensual_h,
    name='Precio Máximo',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=mensual_l.index,
    y=mensual_l,
    name='Precio Mínimo',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Precio mensual Máximo y Mínimo Año 2021')
fig.show()

In [ ]:
names = cycle(['Precio de Apertura','Precio de Cierre','Precio Máximo','Precio Mínimo'])

fig = px.line(y_2021, x=y_2021.datetime, y=[y_2021['open'], y_2021['close'], 
                                          y_2021['high'], y_2021['low']],
             labels={'datetime': 'Año 2021','value':'Precios'})
fig.update_layout(title_text='Gráfico de análisis de Precios Año 2021', font_size=15, font_color='black',legend_title_text='Parámetros de Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

Se pudo observar que el año 2021, los datos muestran cierta estacionalidad, debido a que en los meses de Junio, Julio y agosto bajó drásticamente el precio de esta moneda digital para volver a subir durante el mes de septiembre. Además se puede observar que el precio durante el año 2021 llega a dos máximos históricos ( 64.863 dólares el 14 de abril y $68.789 dólares el 10 de noviembre).Se puede observar su alta volatilidad pero en términos generales fue un buen año para esta moneda digital.

In [ ]:
# Análisis Año 2022
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')
# Ingresar rango del año
y_2022 = df.loc[(df['datetime'] >= '2022-01-01')
                     & (df['datetime'] < '2022-12-31')]
mensual= y_2022.groupby(y_2022['datetime'].dt.strftime('%B'))[['open','close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
mensual = mensual.reindex(new_order, axis=0)
mensual

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=mensual.index,
    y=mensual['open'],
    name='Precio de Apertura',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=mensual.index,
    y=mensual['close'],
    name='Precio de Cierre',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Comparación mensual entre el precio de apertura y cierre Año 2022')
fig.show()

y_2022.groupby(y_2022['datetime'].dt.strftime('%B'))['low'].min()
mensual_h = y_2022.groupby(df['datetime'].dt.strftime('%B'))['high'].max()
mensual_h = mensual_h.reindex(new_order, axis=0)

mensual_l = y_2022.groupby(y_2022['datetime'].dt.strftime('%B'))['low'].min()
mensual_l = mensual_l.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=mensual_h.index,
    y=mensual_h,
    name='Precio Máximo',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=mensual_l.index,
    y=mensual_l,
    name='Precio Mínimo',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Precio mensual Máximo y Mínimo Año 2022')
fig.show()


In [ ]:
names = cycle(['Precio de Apertura','Precio de Cierre','Precio Máximo','Precio Mínimo'])

fig = px.line(y_2022, x=y_2022.datetime, y=[y_2022['open'], y_2022['close'], 
                                          y_2022['high'], y_2022['low']],
             labels={'datetime': 'Año 2022','value':'Precios'})
fig.update_layout(title_text='Gráfico de análisis de Precios Año 2022', font_size=15, font_color='black',legend_title_text='Parámetros de Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

Según el análisis realizado en el año 2022, se observa que el precio tiene una clara tendencia a la baja que comienza a principios de año. Por otro lado, nuevamente su alta volatilidad e incertidumbre como el año anterior vuelve a reaparecer. Durante el mes de Noviembre de este año se detuvo su caída y su precio se mantuvo alrededor de los 16.000 dolares. Por lo que el boom que alcanzó el año anterior, está lejos de volver durante lo que queda de año.

Se puede observar que el precio a fines del año 2020 rompe su tendencia al alza. Se puede observar que durante el año 2021 alcanzó dos máximos históricos y su alta volatilidad durante los dos últimos años. Desde el mes de marzo del año 2022 que la tendencia es bajista y su caída se comienza a desacelerar durante el mes de julio de este año y en el mes de Noviembre su precio se mantuvo en soporte alrededor de los 16.000 USD. Por lo que el boom que alcanzó el año anterior, posiblemente este año ya no se cumpla.

# 3. Selección o reducción de Datos 

Selección de datos o reducción de datos: se define como el proceso en el que los datos relevantes para el análisis, se deciden y recuperan del conglomerado de datos. Se procederá a la selección de los datos para utilizar en el entrenamiento en el modelo LSTM (memoria larga y corto plazo).

#3.1 Selección el conjunto de datos a entrenar 

In [ ]:
# Se realizará una copia del DF original u otro df para entrenar
df1=df_fechas_cierre.copy()
df_entrenar=df1
df_entrenar

#3.1.1 Selección del rango de fechas que se desea evaluar

Dado que el precio de Bitcoin ha fluctuado drásticamente de 400 dólares en el año 2015 a 18.000 dólares en el año 2018 a 3.500 dólares en el año 2019 y al 2021 a $67.000 dólares (valores aproximados), sólo consideraremos 820 días para evitar este tipo de fluctuación en los datos. Además hay que estar atento a los ciclos económicos y análisis de sentimientos de los usuarios y noticias financieras.

In [ ]:
# Se Seleccionara los datos que se desean evaluar (dentro de un año o más). 
df_entrenar= df_entrenar[df_entrenar['datetime'] > '2022-05-01']
stock_cierre = df_entrenar.copy()

print(" Forma del Marco de datos (dataframe):", df_entrenar.shape)
print("Total de datos para la predicción: ",df_entrenar.shape[0])

In [ ]:
df_entrenar

In [ ]:
close='close'

In [ ]:
# Gráfico del Marco de datos a entrenar
fig = px.line(df_entrenar, x=df_entrenar.datetime	, y=df_entrenar.close,labels={'datetime':'Periodos de tiempo Seleccionados','close':'Precios'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Períodos considerado para predecir el precio de cierre de la moneda digital', 
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

# 4. Transformación de los datos 

Transformación de datos: se define como el proceso de transformar los datos a la forma adecuada requerida por el algoritmo computacional usado.

Normalizar o escalar los datos, y muchas veces incluso se deja la información en algún formato vectorial.

# 4.1 Normalización/Estandarización de datos

La normalización es una técnica que se aplica a menudo como parte de la preparación de datos para el aprendizaje automático. El objetivo de la normalización es cambiar los valores de las columnas numéricas en el conjunto de datos para usar una escala común, sin distorsionar las diferencias en los rangos de valores ni perder información.
MinMaxScaler. Para cada valor en una característica, MinMaxScaler subtrae o resta el valor mínimo en la característica y luego lo divide por el rango. El rango es la diferencia entre el máximo original y el mínimo original. MinMaxScaler conserva la forma de la distribución original.

LSTM usa sigmoid and tanh que son sensibles a la magnitud, por lo que los valores deben normalizarse, debido a que pueden existir valores muy distintos y escalar standar es una buena idea por lo general se incluye especialemente en la columna volumen donde hay muchas mas transacciones.

In [ ]:
df_entrenar.dtypes

In [ ]:
# Normalización de los datos usando MinMax Scaler y eliminando la columna de fecha
del df_entrenar['datetime']
scaler = MinMaxScaler(feature_range = (0, 1))
df_entrenar_sc= scaler.fit(df_entrenar)
df_entrenar_sc = scaler.transform(df_entrenar)
print(df_entrenar_sc.shape)

# 5. Minería de datos: Construcción del modelo LSTM

Minería de datos: se define como el uso de técnicas inteligentes sobre los datos para extraer patrones potencialmente útiles.

 ▪ Transforma los datos relevantes de la tarea en patrones.

 ▪ Aquí se decide el objetivo del modelo, ya que se aplica clasificación o agrupamiento.

# 5.1 División para entrenamiento y prueba

Generalmente se considera la regla 75-25. Esto quiere decir, que para la separación de los datos de entrenamiento y prueba, se considera un 75% de los datos para el conjunto de entrenamiento, y un 25% de los datos como conjunto de prueba o testeo.


El conjunto de datos se divide en 75% para entrenamiento y 25% para prueba.
Conjunto de entrenamiento: utilizado para el entrenamiento de modelos.
Conjunto de prueba: utilizado para probar el modelo entrenado.
Asegúrese de que el modelo entrenado nunca antes haya visto el conjunto de datos de prueba.

In [ ]:
#Listas vacías que se completarán con datos de entrenamiento formateados
train_data_X = []
test_data_y = []

In [ ]:
# Número de días pasados ​​que queremos usar para predecir el futuro...
n_ultimo = 15
# Número de días que queremos mirar hacia el futuro en función de los días pasados..
n_futuro = 30  

In [ ]:
# División del conjunto de entrenamiento como 75% y 25% conjunto de prueba para univarible
entrenamiento= 0.75
test= 0.25
porc_entrena=int(len(df_entrenar_sc)*entrenamiento)
porc_prueba=len(df_entrenar_sc)-porc_entrena
train_data_X,test_data_y=df_entrenar_sc[0:porc_entrena,:],df_entrenar_sc[porc_entrena:len(df_entrenar_sc),:1]
print("Entramiento_data: ", train_data_X.shape)
print("Prueba_data: ", test_data_y.shape)

In [ ]:
# Vializamos el conjunto de entrenamiento y prueba
train_data = df_entrenar.iloc[porc_entrena:]
test_data = df_entrenar.iloc[:porc_entrena]
ploteo_linea(train_data[close], test_data[close], 'Prueba', 'Entrenamiento', title='')

In [ ]:
# convertir un array de valores into a dataset matrix
def create_dataset(dataset, time_step=1):
    train_data_X, test_data_y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        train_data_X.append(a)
        test_data_y.append(dataset[i + time_step, 0])
    return np.array(train_data_X), np.array(test_data_y)

In [ ]:
# Número de días pasados ​​que queremos usar para predecir el futuro.
time_step = n_ultimo
X_train, y_train = create_dataset(train_data_X, time_step)
X_test, y_test = create_dataset(test_data_y, time_step)
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)


In [ ]:
# reformar la entrada para que sea[samples, time steps, features] que se requiere para LSTM
# Reformar las matrices 1D,2D a matrices de 3D para alimentar el modelo LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

#5.2 Modelo LSTM

# 5.2.1 Parámetros clave para LSTM con Keras

Algunos parámetros clave para ajustar el modelo LSTM.

* **hidden_layer_sizes:** Debe proporcionar una cantidad de capas ocultas y neuronas para cada capa oculta. Por ejemplo, hidden_layer_sizes – (5,3,3) significa que hay tres capas ocultas y el número de neuronas para la capa uno es 5, para la capa dos es 3 y para la capa tres es 3, respectivamente. El valor predeterminado es (100), es decir, una capa oculta con 100 neuronas.
* **activation:** Esta es la función de activación de una capa oculta; hay cuatro funciones de activación disponibles para su uso; el valor predeterminado es "relu".
     * relu: La función de unidad lineal rectificada, devuelve $f(x) = max(0, x)$
     * logística: La función sigmoidea logística, devuelve f(x) = 1 / (1 + exp(-x)).
     * identidad: activación sin operación, útil para implementar un cuello de botella lineal, devuelve f(x) = x
     * tanh: La función tan hiperbólica, devuelve $f(x) = tanh(x)$.
* **solver:** Esto es para optimizar el peso. Hay tres opciones disponibles, la predeterminada es "adam".
     * adam: Optimizador basado en gradiente estocástico propuesto por Diederik Kingma y Jimmy Ba, que funciona bien para un gran conjunto de datos
     * lbfgs: Pertenece a la familia de métodos cuasi-Newton, funciona bien para conjuntos de datos pequeños
     * sgd: Descenso de gradiente estocástico
* **max_iter:** Este es el número máximo de iteraciones para que el solucionador converja, el valor predeterminado es 200.
* **learning_rate_init:** Esta es la tasa de aprendizaje inicial para controlar el tamaño de paso para actualizar los pesos (solo se aplica a los solucionadores sgd/ adam), el valor predeterminado es 0,001.

In [ ]:
lstm_model = Sequential()
#Entradas
lstm_model.add(LSTM(228,input_shape=(X_train.shape[1],1),activation="relu", return_sequences=True))
#lstm_model.add(Dropout(0.4))
lstm_model.add(LSTM(114, activation='relu', return_sequences=True))
#lstm_model.add(Dropout(0.3))
lstm_model.add(LSTM(57,activation= 'relu', return_sequences=False))
#lstm_model.add(Dropout(0.2))
#Salidas
lstm_model.add(Dense(1))

lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics= ['accuracy'])
lstm_model.summary()

Donde la primera capa de entrada y la dimensión es de 15 días hacia atrás y 228 neuronas "[(None, 15, 228)]" y 209.760 parámetros entrenables.
 
Luego tengo la segunda capa (LSTM): que tiene 114 neuronas (None, 15, 114) y 156.408 parámetros entrenables.
 
Hay a continuación otra red lstm(LSTM) 57 neuronas que tiene 39.216 parámetros entrenables.
Y luego la salida  de la capa densa que tiene 1 variable (None, 1) y 151 parámetros entrenables.
Entonces, en total, con la construcción de las líneas de códigos antes mencionadas, se van a obtener 405,442 parámetros entrenables, que es bastante sorprendente y óptimo.

# 5.2.2 Ajuste del Modelo LSTM

In [ ]:
history = lstm_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=32,verbose=1)

#6. Evaluación del Modelo LSTM

Evaluación de patrones: se define como la interpretación de los resultados, utilizando para esto medidas estándar como reglas de asociación, confianza, entre otros.

Encuentre la puntuación de cada patrón. Utilice resúmenes y visualización para que los datos sean comprensibles para el usuario.


#6.1 Gráfico de Pérdidas(Loss) vs Validación de pérdidas

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'r', label='Pérdida de entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Validación de Pérdidas')
plt.title('Pérdida de entrenamiento y Validación de Pérdidas')
plt.legend(loc=0)
plt.figure()

In [ ]:
#Realización de la Predicción
entrenar_predict=lstm_model.predict(X_train)
test_predict=lstm_model.predict(X_test)
entrenar_predict.shape, test_predict.shape

In [ ]:
# Transformar de nuevo a la forma original
entrenar_predict = scaler.inverse_transform(entrenar_predict)
test_predict = scaler.inverse_transform(test_predict)


In [ ]:
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

#6.2 Métricas de Evaluación del modelo

In [ ]:
evaluacion_metrica(entrenar_predict, original_ytrain, test_predict, original_ytest)

Existen una visión general de las diferentes métricas presentadas, pero nos centraremos en la métrica del R-cuadrado que nos indica la bondad o la aptitud del modelo y además las variables independientes seleccionadas que explican la variabilidad en sus variables dependientes. El Test data R2 score es: 0.73, es decir,el modelo puede explicar el 73% de la varianza. Mientras mayor sea la varianza explicada por el modelo, más cerca están los puntos de los datos de la línea de regresión ajustada.

#6.3 Comparación del precio de cierre original de la Moneda Digital y el precio de cierre previsto

In [ ]:
# Predicciones de cambios de entrenamiento para graficar
look_back=time_step
trainPredictPlot = np.empty_like(df_entrenar_sc)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(entrenar_predict)+look_back, :] = entrenar_predict
print("Predicción de datos Entrenados: ", trainPredictPlot.shape)

# Predicciones de prueba de cambio para graficar
testPredictPlot = np.empty_like(df_entrenar_sc)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(entrenar_predict)+(look_back*2)+1:len(df_entrenar_sc)-1, :] = test_predict
print("Predicción de datos Prueba: ", testPredictPlot.shape)

names = cycle(['Precios de Cierre Original','Entrenamiento Predicción de Precios de Cierre','Test Predicción de Precios de Cierre'])


plotdf = pd.DataFrame({'datetime': stock_cierre['datetime'],
                       'original_cierre': stock_cierre['close'],
                      'train_predicted_cierre': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_cierre': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['datetime'], y=[plotdf['original_cierre'],plotdf['train_predicted_cierre'],
                                          plotdf['test_predicted_cierre']],
              labels={'value':'Precios','datetime': 'Periodo de tiempo'})
fig.update_layout(title_text='Comparación entre el precio de cierre original vs Precio de Cierre pronosticado',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Precios de Cierre')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

#7. Resultados

# 7.1 Salida de los próximos días previstos de 30 días

In [ ]:
x_input=test_data_y[len(test_data_y)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = n_futuro
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} día de entrada {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = lstm_model.predict(x_input, verbose=0)
        #print("{} día de salida {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = lstm_model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Salida de los próximos días previstos: ", len(lst_output))

# 7.2 Trazando los 15 del conjunto de datos y los próximos 30 días

In [ ]:
ult_dias=np.arange(1,time_step+1)
dias_futuro=np.arange(time_step+1,time_step+pred_days+1)
print(ult_dias)
print(dias_futuro)

In [ ]:
temp_mat = np.empty((len(ult_dias)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

ultimo_valor_de_los_dias_originales = temp_mat
prox_valor_de_los_dias_pred = temp_mat

ultimo_valor_de_los_dias_originales[0:time_step+1] = scaler.inverse_transform(df_entrenar_sc[len(df_entrenar_sc)-time_step:]).reshape(1,-1).tolist()[0]
prox_valor_de_los_dias_pred[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'último_valor_de_los_días_originales':ultimo_valor_de_los_dias_originales,
    'próximo_valor_de_los_días_previstos':prox_valor_de_los_dias_pred
})

names = cycle(['Precio de cierre de los últimos 15 días','Precio de cierre previsto para los próximos 30 días'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['último_valor_de_los_días_originales'],
                                                      new_pred_plot['próximo_valor_de_los_días_previstos']],
              labels={'value': 'Precios','index': 'Tiempo (Días)'})
fig.update_layout(title_text='Visualización de la Comparación de los últimos días originales vs los próximos días futuros',
                  plot_bgcolor='seashell', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

# 7.3 Predición Final del Precio de cierre de la moneda digital

In [ ]:
#Trazamos todo el precio de cierre de la Moneda Digital con el próximo período de predicción de n_futuro días
lstmdf=df_entrenar_sc.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Precios de Cierre'])

fig = px.line(lstmdf,labels={'value': 'Precios','index': 'Tiempo (Días)'})
fig.update_layout(title_text='Predición del Precio de cierre (todo el camino + predicción 30 días)',
                  plot_bgcolor='honeydew', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

#8.Conclusiones

Realizar la predicción de precios de una moneda digital o cualquier otro activo financiero, resulta una tarea para nada trivial y más cuando detrás de estos datos se encuentran decisiones y comportamiento de seres humanos. Después de haber realizado nuestro proceso KDD y haber probado el modelo LSTM (memoria larga y corto plazo), debido a su amplio uso en aplicaciones de series de tiempo, como por ejemplo en este caso. Se puede comentar que efectivamente el modelo cree que será de acuerdo a lo que aprendió con los datos históricos, pero al ir ajustando el modelo y realizar las validaciones correspondientes este responde con una predicción razonable. 
 
Cabe destacar que el modelo se puede seguir potenciando, con interacciones que permitan mejorar las medidas de evaluación, minería de datos y los parámetros de configuración. Además de agregar al modelo análisis de sentimientos a través de la plataforma twitter o facebook  o bien análisis de variables multivariantes en el mismo modelo. 
 
La tendencia que predice este modelo LSTM es de tipo conservador, debido a que prevé que el precio de cierre será de 17.500 USD aproximadamente, al 30 de diciembre del año 2022, siempre y cuando exista Ceteris paribus, es decir, que ninguna amenaza externa haga cambiar la tendencia actual.
Por lo tanto, se puede concluir que es un buen momento para comprar Bitcoin, debido a su gran proyección y porque su precio en este momento está en 16.800 USD aproximadamente.
Finalmente los economistas que utilicen la inteligencia artificial y machine learning podrían reducir potencialmente el riesgo y maximizar los rendimientos. Debido a que al predecir con precisión los precios de las monedas digitales, los inversores pueden maximizar los rendimientos y saber cuándo comprar/vender valores.
